In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import Required Libraries

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Adjusting Row Column Settings

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)

In [ ]:
# Loading the Data Set

movie = pd.read_csv('/content/drive/MyDrive/movie_no_year.csv')
rating = pd.read_csv('/content/drive/MyDrive/rating.csv')

In [ ]:
# Merging movie and rating data sets

df = movie.merge(rating, how="left", on="movieId")

In [ ]:
# Preliminary examination of the data set

def check_df(dataframe, head=5):
    print('##################### Shape #####################')
    print(dataframe.shape)
    print('##################### Types #####################')
    print(dataframe.dtypes)
    print('##################### Head #####################')
    print(dataframe.head(head))
    print('##################### Tail #####################')
    print(dataframe.tail(head))
    print('##################### NA #####################')
    print(dataframe.isnull().sum())
    print('##################### Quantiles #####################')
    print(dataframe.describe([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df)

##################### Shape #####################
(20000797, 6)
##################### Types #####################
movieId        int64
title         object
genres        object
userId       float64
rating       float64
timestamp     object
dtype: object
##################### Head #####################
   movieId      title                                       genres  userId  rating            timestamp
0        1  Toy Story  Adventure|Animation|Children|Comedy|Fantasy     3.0     4.0  1999-12-11 13:36:47
1        1  Toy Story  Adventure|Animation|Children|Comedy|Fantasy     6.0     5.0  1997-03-13 17:50:52
2        1  Toy Story  Adventure|Animation|Children|Comedy|Fantasy     8.0     4.0  1996-06-05 13:37:51
3        1  Toy Story  Adventure|Animation|Children|Comedy|Fantasy    10.0     4.0  1999-11-25 02:44:47
4        1  Toy Story  Adventure|Animation|Children|Comedy|Fantasy    11.0     4.5  2009-01-02 01:13:41
##################### Tail #####################
          movieId       

In [ ]:
# We determined how many comments each film received.
comment_counts = pd.DataFrame(df["title"].value_counts())

In [ ]:
comment_counts.head(20)

,count
title,
Pulp Fiction,67310
Forrest Gump,66172
"Shawshank Redemption, The",63366
"Silence of the Lambs, The",63299
Jurassic Park,59715
Star Wars: Episode IV - A New Hope,54502
Braveheart,53769
Terminator 2: Judgment Day,52244
"Matrix, The",51334


In [ ]:
print(df.columns)
print(df['title'].isnull().sum())
df = df[df['title'].notna()]

Index(['movieId', 'title', 'genres', 'userId', 'rating', 'timestamp'], dtype='object')
0


In [ ]:
# Load a random sample of the rating dataset
rating_sample = rating.sample(frac=0.1, random_state=42)  # 10% sample, adjust as needed

# Merge with the movie data
df_sample = movie.merge(rating_sample, how="left", on="movieId")

# Filter for common movies
comment_counts = df_sample['title'].value_counts()
rare_movies = comment_counts[comment_counts <= 1000].index
common_movies_sample = df_sample[~df_sample['title'].isin(rare_movies)]

# Create the User-Movie DataFrame
user_movie_df = common_movies_sample.pivot_table(index="userId", columns="title", values="rating")


In [ ]:
user_movie_df.head()

title,10 Things I Hate About You,12 Angry Men,2001: A Space Odyssey,28 Days Later,300,A.I. Artificial Intelligence,"Abyss, The",Ace Ventura: Pet Detective,Ace Ventura: When Nature Calls,Addams Family Values,"African Queen, The",Air Force One,Airplane!,Aladdin,Alice in Wonderland,Alien,Alien: Resurrection,Aliens,Alien³ (a.k.a. Alien 3),Almost Famous,Amadeus,"Amelie (Fabuleux destin d'Amélie Poulain, Le)",American Beauty,American History X,American Pie,"American President, The",Animal House,Annie Hall,Antz,Apocalypse Now,Apollo 13,Arachnophobia,Armageddon,Army of Darkness,As Good as It Gets,Austin Powers: International Man of Mystery,Austin Powers: The Spy Who Shagged Me,Babe,Back to the Future,Back to the Future Part II,Back to the Future Part III,Bad Boys,Basic Instinct,Batman,Batman Begins,Batman Forever,Batman Returns,"Beautiful Mind, A",Beauty and the Beast,Beetlejuice,Being John Malkovich,Best in Show,Beverly Hills Cop,Beverly Hills Cop III,Big,Big Fish,"Big Lebowski, The","Birdcage, The","Birds, The",Black Hawk Down,Blade,Blade Runner,"Blair Witch Project, The",Blazing Saddles,"Blues Brothers, The",Boogie Nights,"Boot, Das (Boat, The)","Bourne Identity, The","Bourne Supremacy, The","Bourne Ultimatum, The",Bowling for Columbine,Braveheart,Brazil,"Breakfast Club, The","Bridge on the River Kwai, The",Bridget Jones's Diary,Broken Arrow,"Bug's Life, A",Butch Cassidy and the Sundance Kid,"Cable Guy, The",Cape Fear,Casablanca,Casino,Casino Royale,Casper,Cast Away,Catch Me If You Can,Charlie's Angels,Chasing Amy,Chicken Run,Chinatown,Chocolat,"Christmas Story, A",Citizen Kane,City Slickers II: The Legend of Curly's Gold,City of God (Cidade de Deus),Clear and Present Danger,Clerks,"Client, The",Cliffhanger,"Clockwork Orange, A",Close Encounters of the Third Kind,Clueless,Cocoon,Con Air,Coneheads,Congo,Conspiracy Theory,Contact,Cool Hand Luke,Crash,Crimson Tide,Crocodile Dundee,"Crouching Tiger, Hidden Dragon (Wo hu cang long)","Crow, The","Crying Game, The",Dances with Wolves,Dark City,"Dark Knight, The",Dave,Dead Man Walking,Dead Poets Society,Deep Impact,"Deer Hunter, The",Demolition Man,"Departed, The",Desperado,"Devil's Advocate, The",Die Hard,Die Hard 2,Die Hard: With a Vengeance,Dirty Dancing,Disclosure,Dogma,Donnie Brasco,Donnie Darko,Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb,Dragonheart,Dumb & Dumber (Dumb and Dumber),Dune,E.T. the Extra-Terrestrial,Ed Wood,Edward Scissorhands,Election,Enemy of the State,"English Patient, The",Eraser,Erin Brockovich,Eternal Sunshine of the Spotless Mind,Executive Decision,"Exorcist, The",Eyes Wide Shut,Face/Off,Fantasia,Fargo,Father of the Bride Part II,Ferris Bueller's Day Off,"Few Good Men, A",Field of Dreams,"Fifth Element, The",Fight Club,Finding Nemo,"Firm, The",First Knight,"Fish Called Wanda, A","Fly, The",Forrest Gump,Four Weddings and a Funeral,French Kiss,From Dusk Till Dawn,"Fugitive, The",Full Metal Jacket,"Full Monty, The",Galaxy Quest,"Game, The",Gattaca,Get Shorty,Ghost,Ghostbusters (a.k.a. Ghost Busters),Gladiator,Glory,"Godfather, The","Godfather: Part II, The","Godfather: Part III, The",GoldenEye,Goldfinger,Gone with the Wind,"Good Morning, Vietnam",Good Will Hunting,"Good, the Bad and the Ugly, The (Buono, il brutto, il cattivo, Il)",Goodfellas,"Goonies, The","Graduate, The",Grease,"Great Escape, The","Green Mile, The",Gremlins,Grosse Pointe Blank,Groundhog Day,Grumpier Old Men,Hamlet,Happy Gilmore,Harry Potter and the Chamber of Secrets,Harry Potter and the Goblet of Fire,Harry Potter and the Prisoner of Azkaban,Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone),Heat,Heathers,High Fidelity,Highlander,Home Alone,"Honey, I Shrunk the Kids",Hoop Dreams,Hot Shots! Part Deux,"Hudsucker Proxy, The","Hunchback of Notre Dame, The","Hunt for Red October, The","I, Robot",I.Q.,Ice Age,In the Line of Fire,Inception,"Incredibles, The",Independence Day (a.k.a. ID4),Indiana Jones and the Last Crusade,Indiana Jones and the

In [ ]:
user_movie_df.shape

(126374, 470)

In [ ]:
user_movie_df.columns

Index(['10 Things I Hate About You', '12 Angry Men', '2001: A Space Odyssey', '28 Days Later', '300', 'A.I. Artificial Intelligence', 'Abyss, The', 'Ace Ventura: Pet Detective', 'Ace Ventura: When Nature Calls', 'Addams Family Values',
       ...
       'Wild Wild West', 'William Shakespeare's Romeo + Juliet', 'Willy Wonka & the Chocolate Factory', 'Witness', 'Wizard of Oz, The', 'X-Files: Fight the Future, The', 'X-Men', 'X2: X-Men United', 'You've Got Mail', 'Young Frankenstein'], dtype='object', name='title', length=470)

In [ ]:
movie_name1 = "Toy Story"

In [ ]:
if movie_name1 in user_movie_df.columns:
    movie_name = user_movie_df[movie_name1]
else:
    print(f"'{movie_name1}' not found in user_movie_df columns.")


# **Surprise library implementation**

### **Use numpy, pandas, and sklearn**   

In [ ]:
# Install necessary dependencies with forced reinstallation
!pip install --ignore-installed itsdangerous blinker
!pip install --no-deps flask

from flask import Flask, render_template, request, send_from_directory
import pandas as pd
import os
import shutil
from google.colab import drive
from google.colab.output import eval_js
import re
from urllib.parse import quote

import requests

# TMDb API details
TMDB_API_KEY = "cded237d0baeec6b725cacea5d0c4a2c"
TMDB_BASE_URL = "https://api.themoviedb.org/3"
TMDB_IMAGE_BASE_URL = "https://image.tmdb.org/t/p/w500"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.3 MB/s eta 0:00:00


In [ ]:
# Define paths
POSTER_PATH = '/content/drive/MyDrive/posters/posters'  # Google Drive path for posters
GOOGLE_DRIVE_TEMPLATE_PATH = '/content/drive/MyDrive/templates'  # Path to templates on Google Drive

# Initialize Flask app with the template folder in Google Drive
app = Flask(__name__, template_folder=GOOGLE_DRIVE_TEMPLATE_PATH)

# Assuming df_sample and user_movie_df are defined somewhere
# df_sample = pd.read_csv('path_to_movies_csv')  # Path to movie data CSV
# user_movie_df = pd.read_csv('path_to_user_movie_csv')  # Path to user movie data CSV

def clean_title(title):
    return title.strip().lower()

def get_movie_info(movie_title):
    # Look for movie info in the dataset
    movie_info_df = df_sample[df_sample['title'].str.lower() == movie_title.lower()]

    # Attempt to load the local poster if available in Google Drive
    if not movie_info_df.empty:
        movie_info = movie_info_df.iloc[0]
        poster_filename = f"{movie_title.replace(' ', '_')}.jpg"
        google_drive_poster_path = os.path.join(POSTER_PATH, poster_filename)

        # Check if the poster exists in Google Drive; if not, use TMDb
        if os.path.isfile(google_drive_poster_path):
            return {
                'title': movie_info['title'],
                'genres': movie_info['genres'],
                'poster': f'/posters/{poster_filename}'  # The route now points to Google Drive
            }

    # Fetch data from TMDb if not in local or missing the local poster
    response = requests.get(
        f"{TMDB_BASE_URL}/search/movie",
        params={"api_key": TMDB_API_KEY, "query": movie_title}
    )
    data = response.json()
    if data["results"]:
        movie_data = data["results"][0]
        poster_url = f"{TMDB_IMAGE_BASE_URL}{movie_data['poster_path']}" if movie_data.get('poster_path') else None
        return {
            'title': movie_data['title'],
            'genres': "Genre data unavailable",
            'poster': poster_url or '/path/to/placeholder.jpg'
        }
    else:
        print(f"Movie '{movie_title}' not found on TMDb.")
        return None

def recommend_movies(movie_name):
    if movie_name not in user_movie_df.columns:
        print(f"Movie '{movie_name}' not found in user_movie_df")
        return []

    movie_series = user_movie_df[movie_name]
    similar_movies = user_movie_df.corrwith(movie_series).sort_values(ascending=False).head(20)

    recommended_movies = []
    for title in similar_movies.index:
        if title != movie_name:
            info = get_movie_info(title)
            if info:
                recommended_movies.append(info)

    return recommended_movies

@app.route('/', methods=['GET', 'POST'])
def index():
    recommendations = []
    movie_info = None
    if request.method == 'POST':
        movie_name = request.form.get('movie_name')
        print(f"Received movie name: {movie_name}")

        movie_info = get_movie_info(movie_name)
        print(f"Movie info: {movie_info}")

        if movie_info:
            recommendations = recommend_movies(movie_name)
            print(f"Recommendations: {recommendations}")

    return render_template('index.html', movie_info=movie_info, recommendations=recommendations)

# Route to serve posters directly from Google Drive
@app.route('/posters/<path:filename>')
def send_poster(filename):
    return send_from_directory(POSTER_PATH, filename)

if __name__ == '__main__':
    print(eval_js("google.colab.kernel.proxyPort(5000)"))
    app.run(port=5000)

https://kg51p008nya-496ff2e9c6d22116-5000-colab.googleusercontent.com/
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [06/Nov/2024 01:11:27] "GET /?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Nov/2024 01:11:27] "GET /favicon.ico?authuser=0 HTTP/1.1" 404 -


Received movie name: Titanic
Movie info: {'title': 'Titanic', 'genres': 'Drama|Romance', 'poster': '/posters/Titanic.jpg'}


INFO:werkzeug:127.0.0.1 - - [06/Nov/2024 02:01:35] "POST /?authuser=0 HTTP/1.1" 200 -


Recommendations: [{'title': 'Legends of the Fall', 'genres': 'Drama|Romance|War|Western', 'poster': '/posters/Legends_of_the_Fall.jpg'}, {'title': 'Honey, I Shrunk the Kids', 'genres': 'Genre data unavailable', 'poster': 'https://image.tmdb.org/t/p/w500/1Xebz5V4UQchh49NXQ0n6FFLYQb.jpg'}, {'title': 'Crocodile Dundee', 'genres': 'Adventure|Comedy', 'poster': '/posters/Crocodile_Dundee.jpg'}, {'title': 'Speed', 'genres': 'Action|Romance|Thriller', 'poster': '/posters/Speed.jpg'}, {'title': 'Nell', 'genres': 'Drama', 'poster': '/posters/Nell.jpg'}, {'title': 'Addams Family Values', 'genres': 'Genre data unavailable', 'poster': 'https://image.tmdb.org/t/p/w500/sdxT2VjVSx9DRicwnuECUdBHeE7.jpg'}, {'title': 'Deep Impact', 'genres': 'Drama|Sci-Fi|Thriller', 'poster': '/posters/Deep_Impact.jpg'}, {'title': 'E.T. the Extra-Terrestrial', 'genres': 'Children|Drama|Sci-Fi', 'poster': '/posters/E.T._the_Extra-Terrestrial.jpg'}, {'title': "What's Eating Gilbert Grape", 'genres': 'Genre data unavailabl

...

INFO:werkzeug:127.0.0.1 - - [06/Nov/2024 02:15:46] "GET /posters/Ice_Age.jpg?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Nov/2024 02:15:46] "GET /posters/Legends_of_the_Fall.jpg?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Nov/2024 02:15:46] "GET /posters/Titanic.jpg?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Nov/2024 02:15:47] "GET /posters/Juno.jpg?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Nov/2024 02:15:47] "GET /posters/Rain_Man.jpg?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Nov/2024 02:15:47] "GET /posters/Inception.jpg?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Nov/2024 02:15:47] "GET /posters/My_Big_Fat_Greek_Wedding.jpg?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Nov/2024 02:15:47] "GET /posters/Wag_the_Dog.jpg?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Nov/2024 02:15:47] "GET /posters/Wild_Wild_West.jpg?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 

Received movie name: Inception
Movie info: {'title': 'Inception', 'genres': 'Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX', 'poster': '/posters/Inception.jpg'}


/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
INFO:werkzeug:127.0.0.1 - - [06/Nov/2024 02:16:09] "POST /?authuser=0 HTTP/1.1" 200 -


Recommendations: [{'title': 'The Maltese Falcon', 'genres': 'Genre data unavailable', 'poster': 'https://image.tmdb.org/t/p/w500/bf4o6Uzw5wqLjdKwRuiDrN1xyvl.jpg'}, {'title': 'The Russian Specialist', 'genres': 'Genre data unavailable', 'poster': 'https://image.tmdb.org/t/p/w500/imt2pLZmvOiaH1HteyToJVjJGcH.jpg'}, {'title': 'Cool Hand Luke', 'genres': 'Genre data unavailable', 'poster': 'https://image.tmdb.org/t/p/w500/4ykzTiHKLamh3eZJ8orVICtU2Jp.jpg'}, {'title': 'Nell', 'genres': 'Drama', 'poster': '/posters/Nell.jpg'}, {'title': 'Beverly Hills Cop III', 'genres': 'Action|Comedy|Crime|Thriller', 'poster': '/posters/Beverly_Hills_Cop_III.jpg'}, {'title': 'The Postman', 'genres': 'Genre data unavailable', 'poster': 'https://image.tmdb.org/t/p/w500/cUaCpjVDefYShKyLmkcDsiPaBHn.jpg'}, {'title': 'Butch Cassidy and the Sundance Kid', 'genres': 'Action|Western', 'poster': '/posters/Butch_Cassidy_and_the_Sundance_Kid.jpg'}, {'title': 'Like Water for Chocolate', 'genres': 'Genre data unavailable'

INFO:werkzeug:127.0.0.1 - - [06/Nov/2024 02:16:09] "GET /posters/Inception.jpg?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Nov/2024 02:16:09] "GET /posters/Legends_of_the_Fall.jpg?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Nov/2024 02:16:09] "GET /posters/Nell.jpg?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Nov/2024 02:16:09] "GET /posters/Beverly_Hills_Cop_III.jpg?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Nov/2024 02:16:10] "GET /posters/Congo.jpg?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Nov/2024 02:16:10] "GET /posters/Star_Trek_III:_The_Search_for_Spock.jpg?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Nov/2024 02:16:10] "GET /posters/Conspiracy_Theory.jpg?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Nov/2024 02:16:10] "GET /posters/Butch_Cassidy_and_the_Sundance_Kid.jpg?authuser=0 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Nov/2024 02:16:10] "GET /favicon.ico?authuser=0 